# Modelos de Recomendação Baseados em Similaridade de Conteúdo

In [1]:
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.spatial.distance import cdist
from IPython.display import display
from sklearn.metrics import davies_bouldin_score, silhouette_score
import sys
sys.path.append('../auxiliarScripts')
from dataset_reader import SpotifyPlaylistDataset

### Carrega e Normaliza os Dados

In [2]:
tracks = pd.read_csv('../tracks/tracks.csv', index_col='id')
tracks

,name,duration_ms,artists,explicit,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature
id,,,,,,,,,,,,,,,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,52213,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War...",t,0,0.004530,0.324,0.971,0.012800,0,0.3340,-3.168,1,0.0548,95.721,4
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,promise [interlude],88888,"{""(5b0XQwyoJBFd3MwL2YxEPO,Nohidea)""}",f,0,0.833000,0.545,0.495,0.629000,11,0.3860,-13.538,1,0.1020,62.631,4
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,Requiem - Alex Vidal Remix,564720,"{""(6WrPC9g41hy7LUnpxdy6u7,\""Alex Vidal\"")"",""(2...",f,0,0.033200,0.695,0.592,0.892000,0,0.5790,-9.588,1,0.0345,125.008,4
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,Kaké,245356,"{""(44DhAhLWsSFvSqVXDUruVE,\""Young Paris\"")""}",t,0,0.000440,0.450,0.629,0.895000,7,0.1080,-7.059,1,0.1460,193.228,3
spotify:track:16axQgz02GFMiU2xKMT4qJ,F the Bull$h1t - Original Mix,315773,"{""(2ohlvFf9PBsDELdRstPtlP,\""Ferry Corsten\"")""}",f,0,0.000927,0.846,0.707,0.903000,6,0.0666,-5.949,0,0.1270,127.970,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,Waking Up The Dawn,294333,"{""(7beSXA7nBwZ5xJ8jUa77ZL,\""Michael English\"")""}",f,2,0.109000,0.538,0.609,0.000006,5,0.0948,-7.392,1,0.0275,87.960,4
spotify:track:1nvTse7nkCr8tO8LOAell4,Here Comes the Star,202733,"{""(48YxSlb23RAaCd4RyHcV9V,\""Herman's Hermits\"")""}",f,0,0.698000,0.339,0.325,0.000005,2,0.5550,-11.445,1,0.0252,91.286,4
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,You're With Me,293546,"{""(0WTXG5dSB0wpykAqPpEmFl,UnchainD)""}",f,0,0.018600,0.482,0.443,0.000345,4,0.1100,-8.622,1,0.0276,144.962,4


#### Para pegar uma música: tracks.loc[id]
#### Para pesquisar o nome de uma música e pegar seu id: tracks.index[tracks['name'] == "nome música"]

In [3]:
# Músicas que vão simular as músicas do usuário a serem recomendadas
classical_music_i = tracks.index[tracks['name'] == 'Symphony No.5 in C minor, Op.67: 1. Allegro con brio'][0]
rap_music = tracks.index[tracks['name'] == 'Yoshi City'][0]
anime_music = tracks.index[tracks['name'] == 'GO!!!'][0]
anime_music2 = 'spotify:track:1hi0P1Y2he1p09xnUccuAN'
kaio_music = tracks.index[tracks['name'] == 'Wrist Cry'][0]
kaio_music2 = tracks.index[tracks['name'] == 'Reptilia'][0]
eletronic_music = 'spotify:track:47qYqGPgMTh3l1PRKBHEfI' #unity FatRat

#### Deixa apenas as features que serão utilizadas e separa um dataframe com nome e artistas para poderem ser retornados no final

In [4]:
# guarda o id e nome
music_ids = tracks[['name', 'artists']]

# remove colunas que não serão utilizadas
tracks.drop(['name', 'artists', 'explicit', 'time_signature', 'mode', 'key', 'duration_ms', 'popularity'], axis=1, inplace=True)
music_ids

,name,artists
id,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War..."
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,promise [interlude],"{""(5b0XQwyoJBFd3MwL2YxEPO,Nohidea)""}"
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,Requiem - Alex Vidal Remix,"{""(6WrPC9g41hy7LUnpxdy6u7,\""Alex Vidal\"")"",""(2..."
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,Kaké,"{""(44DhAhLWsSFvSqVXDUruVE,\""Young Paris\"")""}"
spotify:track:16axQgz02GFMiU2xKMT4qJ,F the Bull$h1t - Original Mix,"{""(2ohlvFf9PBsDELdRstPtlP,\""Ferry Corsten\"")""}"
...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,Waking Up The Dawn,"{""(7beSXA7nBwZ5xJ8jUa77ZL,\""Michael English\"")""}"
spotify:track:1nvTse7nkCr8tO8LOAell4,Here Comes the Star,"{""(48YxSlb23RAaCd4RyHcV9V,\""Herman's Hermits\"")""}"
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,You're With Me,"{""(0WTXG5dSB0wpykAqPpEmFl,UnchainD)""}"


In [5]:
tracks

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,0.004530,0.324,0.971,0.012800,0.3340,-3.168,0.0548,95.721
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,0.833000,0.545,0.495,0.629000,0.3860,-13.538,0.1020,62.631
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,0.033200,0.695,0.592,0.892000,0.5790,-9.588,0.0345,125.008
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,0.000440,0.450,0.629,0.895000,0.1080,-7.059,0.1460,193.228
spotify:track:16axQgz02GFMiU2xKMT4qJ,0.000927,0.846,0.707,0.903000,0.0666,-5.949,0.1270,127.970
...,...,...,...,...,...,...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,0.109000,0.538,0.609,0.000006,0.0948,-7.392,0.0275,87.960
spotify:track:1nvTse7nkCr8tO8LOAell4,0.698000,0.339,0.325,0.000005,0.5550,-11.445,0.0252,91.286
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,0.018600,0.482,0.443,0.000345,0.1100,-8.622,0.0276,144.962


### Função de normalização usando fórmula do MinMaxScaler do SKLearn 
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [6]:
# função que recebe um pandas Series e a escala sem copiar memória
def scaler(X, min_=None, max_=None):
    if not min_:
        min_ = X.min()
    if not max_:
        max_ = X.max()
    return X.map(lambda x: (x - min_) / (max_ - min_))

### Normalização dos dados que não estão normalizados

In [7]:
#tracks['duration_ms'] = scaler(tracks['duration_ms'])
#tracks['popularity'] = scaler(tracks['popularity'], min_=0, max_=100)
#tracks['key'] = scaler(tracks['key'])
tracks['loudness'] = scaler(tracks['loudness'], min_=-60, max_=0)
tracks['tempo'] = scaler(tracks['tempo'])

In [8]:
tracks

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,0.004530,0.324,0.971,0.012800,0.3340,0.875375,0.0548,0.382904
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,0.833000,0.545,0.495,0.629000,0.3860,0.715648,0.1020,0.250537
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,0.033200,0.695,0.592,0.892000,0.5790,0.776489,0.0345,0.500058
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,0.000440,0.450,0.629,0.895000,0.1080,0.815443,0.1460,0.772952
spotify:track:16axQgz02GFMiU2xKMT4qJ,0.000927,0.846,0.707,0.903000,0.0666,0.832540,0.1270,0.511907
...,...,...,...,...,...,...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,0.109000,0.538,0.609,0.000006,0.0948,0.810314,0.0275,0.351858
spotify:track:1nvTse7nkCr8tO8LOAell4,0.698000,0.339,0.325,0.000005,0.5550,0.747886,0.0252,0.365163
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,0.018600,0.482,0.443,0.000345,0.1100,0.791368,0.0276,0.579878


## Criando Modelo de Similaridade

In [9]:
class Similarity:
    def __init__(self, distance='euclidean'):
        self.distance = distance
        self.tracks = None
        
    def fit(self, tracks, id_tracks):
        self.tracks = tracks
        self.id_tracks = id_tracks
    
    def recommend(self, user_tracks, n_tracks=5):
        # cria a música média
        mean_track = np.mean(user_tracks, axis=0)
        
        # calcula distancias e pega os indices das k músicas com menores distâncias
        dists = cdist([mean_track], self.tracks, self.distance)[0]
        k_arg_dists = np.argpartition(dists, n_tracks+1)[:n_tracks+1]
        
        k_ids = self.id_tracks.iloc[k_arg_dists]
        k_info = self.tracks.iloc[k_arg_dists]
        k_dists = pd.DataFrame(dists[k_arg_dists], columns=['Distâncias'], index=self.tracks.index[k_arg_dists])
        
        # retorna um dataframe com informação do nome, artistas, distância e features ordenadas pela distância
        return pd.concat([k_dists, k_ids, k_info], axis=1).sort_values(by=['Distâncias'])
        

### Teste de Recomendador para Música Única

In [10]:
sim2 = Similarity(distance='euclidean')
sim2.fit(tracks, music_ids)
sim2.recommend([tracks.loc[classical_music_i]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:4Znak3UnsXjRYQDSK0oP6V,0.000000,"Symphony No.5 in C minor, Op.67: 1. Allegro co...","{""(2wOqMjp9TyABvtHdOSOTUS,\""Ludwig van Beethov...",0.900,0.309,0.307,0.745,0.0929,0.764198,0.0443,0.402445
spotify:track:061pN2sHhLNI3vAPIHVn2R,0.053293,Stranger On The Shore,"{""(5bdgb81IJMXag724quaNSA,\""Acker Bilk\"")"",""(4...",0.940,0.319,0.322,0.742,0.1040,0.740200,0.0359,0.390764
spotify:track:1pKwem2d9YXLd4LUParsid,0.055306,THEME FROM LOVE STORY,"{""(1sLpCr5xThZkGHLw4H0mNE,\""101 Strings Orches...",0.900,0.289,0.315,0.716,0.1050,0.728632,0.0288,0.412557
spotify:track:1mBIJRcbAec5EkveeYrPCa,0.065940,I Got Lost,"{""(267VY6GX5LyU5c9M85ECZQ,\""Dinosaur Jr.\"")""}",0.875,0.354,0.330,0.748,0.0942,0.738983,0.0264,0.388224
spotify:track:5GA90eh24Ro95k9tywfuHo,0.069033,Unless a Grain of Wheat,"{""(3PF6OyABdvKssByiQ8Imn9,\""Forbes Ministry of...",0.886,0.359,0.292,0.738,0.1310,0.756327,0.0276,0.404209
spotify:track:42B96jvtflrB3Xo7rcJnl2,0.069542,American Civil War Fantasy,"{""(6lxjjQH0jBo0mEHO5WxFnS,\""United States Coas...",0.910,0.319,0.303,0.773,0.0794,0.741124,0.0388,0.458044


In [11]:
sim3 = Similarity(distance='euclidean')
sim3.fit(tracks, music_ids)
sim3.recommend([tracks.loc[rap_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:5Tou8lYopj9qCIFBFBomZ5,0.000000,Yoshi City,"{""(67lytN32YpUxiSeWlKfHJ3,\""Yung Lean\"")""}",0.476,0.546,0.530,0.000000,0.0955,0.784637,0.211,0.512863
spotify:track:0kDEzhgtVZfYMVm7VR7f03,0.060607,Hero,"{""(0fLP4yWtS7wE0S0QbI4IYG,\""Chan Dizzy\"")""}",0.436,0.563,0.539,0.000000,0.1220,0.796189,0.232,0.492210
spotify:track:2CIzsTwVjIWMh64Nw6S3Lp,0.061630,Heartbroken Forever,"{""(5WF5qT6fSfyd3RC2p8qpEX,\""Tender Forever\"")""}",0.518,0.534,0.509,0.000000,0.1130,0.786131,0.217,0.479625
spotify:track:4vh8S4hTWAz3DHjysBo0ai,0.061667,Cocaine,"{""(4YUcg2ee9TxMNdjhAxzZBF,Ashh)"",""(44JOYPmzHzT...",0.498,0.584,0.506,0.000028,0.1110,0.772084,0.181,0.511899
spotify:track:4va49u7BqBLAmpu7WnyuQ1,0.064300,Heartbreaker,"{""(4mwXUEKaW4ftbncf9Hi58l,Tank)""}",0.496,0.510,0.500,0.000000,0.0637,0.805677,0.217,0.519827
spotify:track:00VWUa96Clte4uNEv963vn,0.070310,Évite la,"{""(1cA379ILgR07m7XQVXyP6Q,Sneazzy)"",""(5olcgTze...",0.442,0.603,0.538,0.000049,0.1010,0.786347,0.232,0.513463


In [12]:
sim4 = Similarity(distance='euclidean')
sim4.fit(tracks, music_ids)
sim4.recommend([tracks.loc[anime_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:0zFRHGYU1zP6q45qbils3z,0.000000,GO!!!,"{""(3w2HqkKa6upwuXEULtGvnY,FLOW)""}",0.0508,0.604,0.978,0.000000,0.228,0.891595,0.0573,0.536584
spotify:track:5X31etfs08Q3H7WenRHWoc,0.038166,UNBALANCE,"{""(7Jo4alROJu9Smj8IZahiP1,Dios/signalP)""}",0.0471,0.590,0.956,0.004540,0.242,0.881752,0.0395,0.548101
spotify:track:5XuGyBKKFrCoz4iwvm7k7s,0.042463,I Hope,"{""(6KhH771vq2X2Aom91nNzpZ,FTISLAND)""}",0.0580,0.590,0.976,0.000000,0.197,0.892426,0.0641,0.559893
spotify:track:5z8ynRCdK17BWNLxL8rrfZ,0.043371,Take It Like a Man,"{""(72OaDtakiy6yFqkt4TsiFt,Cher)""}",0.0678,0.613,0.950,0.000000,0.243,0.889977,0.0723,0.520019
spotify:track:62pM9hM5FdqPuUKkZTmENp,0.043751,fanfare,"{""(0udR1QMfgT37SQnL1QClTU,\""Bullet Train\"")""}",0.0426,0.613,0.943,0.000000,0.208,0.883323,0.0651,0.540000
spotify:track:4618Zrqjfr2nqeXibDUIFJ,0.044536,Make me fly - The Out In Space Story Continues,"{""(40edmg1pVhEKklKU7BCo1n,\""Timur Rodriguez\"")""}",0.0263,0.620,0.967,0.000008,0.250,0.889069,0.0475,0.516095


In [13]:
sim5 = Similarity(distance='euclidean')
sim5.fit(tracks, music_ids)
sim5.recommend([tracks.loc[kaio_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:15TJBhID2NsMMp2etO0B03,0.000000,Wrist Cry,"{""(2xvtxDNInKDV4AvGmjw6d1,Bladee)""}",0.366,0.634,0.428,0.003270,0.280,0.738906,0.0284,0.439895
spotify:track:5X7yF7u1fVGS4EyekX6ewM,0.025748,"Miami, My Amy","{""(0bc7dhY6YXL35m1G0jVwuW,\""Keith Whitley\"")""}",0.354,0.646,0.441,0.000002,0.290,0.746592,0.0272,0.433999
spotify:track:1OJpQrusmejmwnvsFQqF8c,0.035643,Annie,"{""(05Pp7SeRkvE236OxUF2qmo,Yarn)""}",0.341,0.645,0.443,0.000027,0.290,0.752599,0.0276,0.440767
spotify:track:3zyUs7o9iKVhb1xuSSi0t8,0.038462,Compañero de Mi Alma,"{""(3NUe53SAgcR19GtgfSKTpW,\""Óscar Medina\"")""}",0.374,0.646,0.413,0.000000,0.265,0.711628,0.0302,0.431942
spotify:track:0Avs2CCYWfhanvCMc5qtnK,0.040308,Friends - Acoustic,"{""(6jO0ArPsbU6gg4GehuCjoF,\""Eliza and the Bear...",0.331,0.633,0.429,0.000006,0.299,0.743912,0.0293,0.440395
spotify:track:0DT4o6GdCw5Uxqb5EDIqSf,0.043744,Don't Try To Explain,"{""(6iDaoPZVgxrTkndDCisX8F,\""Keb' Mo'\"")""}",0.386,0.649,0.452,0.000469,0.278,0.762904,0.0393,0.442339


In [14]:
sim6 = Similarity(distance='euclidean')
sim6.fit(tracks, music_ids)
sim6.recommend([tracks.loc[kaio_music2]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:2hmibAtdObO8F4tnhLENuQ,0.000000,Reptilia,"{""(0epOFNiUfyON9EYx7Tpr6V,\""The Strokes\"")""}",0.000597,0.488,0.650,0.714,0.1010,0.845463,0.0336,0.632077
spotify:track:79NvOn5z1gmBhZMvAdaZzv,0.039079,L'innocent,"{""(0iF2gedsSjoVHRUh5vXSdu,\""Cirque du Soleil\"")""}",0.013300,0.496,0.638,0.687,0.0973,0.827134,0.0296,0.640021
spotify:track:1Dc4H1XvXdsRqrD6WZxYKZ,0.056278,Lower Down,"{""(4zxyqU3xB0RrPX9rL9RNln,\""Lo Tom\"")""}",0.000043,0.483,0.699,0.714,0.1210,0.853781,0.0292,0.616192
spotify:track:6LVbyUVhbhhNN7uc2bqEqg,0.060271,Strip The Soul,"{""(5NXHXK6hOCotCF8lvGM1I0,\""Porcupine Tree\"")""}",0.001920,0.537,0.670,0.716,0.1150,0.830538,0.0470,0.616992
spotify:track:3JxGBrNcvunLiznL1JJzZN,0.066181,Is This Real?,"{""(0sTTw3dw3EA0c7NaZnrJd2,Wipers)""}",0.003300,0.456,0.695,0.714,0.1310,0.839302,0.0324,0.651678
spotify:track:1QlweZy3kMgha3UlkQbjMS,0.066457,Take This Lying Down,"{""(2VS0tRcCbiMkelzGoZo9XP,\""The Lucksmiths\"")""}",0.002320,0.479,0.631,0.711,0.0732,0.797268,0.0264,0.603527


In [15]:
sim7 = Similarity(distance='euclidean')
sim7.fit(tracks, music_ids)
sim7.recommend([tracks.loc[eletronic_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:47qYqGPgMTh3l1PRKBHEfI,0.000000,Unity,"{""(3OKg7YbOIatODzkRIbLJR4,TheFatRat)""}",0.004240,0.707,0.880,0.03790,0.0534,0.874713,0.0317,0.420034
spotify:track:6Gd317x1MJZ378T6xbCpGI,0.041626,Jacuzzi Jam - Original Mix,"{""(3uMZB1LatgHJCRe7DBtgiK,Syndaesia)""}",0.000565,0.728,0.873,0.03190,0.0659,0.872125,0.0569,0.439915
spotify:track:5HRnngZD7Xgko5swVsuYIk,0.043683,How I Feel,"{""(1fwbNA9Ymk7hPxcxx67paW,\""Brown and Gammon\"")""}",0.002050,0.698,0.889,0.03740,0.0418,0.856692,0.0541,0.447963
spotify:track:3MYStY0UdBrk01XUDfYON1,0.046224,Phoenix - DJ Laszlo Remix,"{""(188VINPQh5dyNfLUkevKqf,\""Olivia Holt\"")"",""(...",0.029200,0.697,0.899,0.01050,0.0407,0.881999,0.0365,0.412001
spotify:track:6DlwyiBUy2b0JQWd6CLAQa,0.046876,Love To Love You Baby - Giorgio Moroder Feat. ...,"{""(2eogQKWWoohI3BSnoG7E2U,\""Donna Summer\"")"",""...",0.019000,0.710,0.878,0.00499,0.0623,0.853303,0.0458,0.432158
spotify:track:5TYIzi4Oa1fiRpDX0mklLN,0.047079,Hunter - Made in June Remix,"{""(4sTQVOfp9vEMCemLw50sbu,Galantis)"",""(0EJYJn2...",0.005150,0.719,0.865,0.00709,0.0827,0.871987,0.0356,0.424062


### Teste de Recomendador para Playlist de Músicas

In [16]:
class ContentBasedEvaluator:
    '''
    Esse Evaluator pretende avaliar o recomendador.
    Recebe um modelo, recomenda para as tracks de treino e retorna as distâncias que esse modelo encontrou
    para os dados de teste. Então, se temos duas músicas que consideramos parecidas, podemos colocar
    uma para ser de treino e outra de teste e então ele vai retornar a distância delas. Se for pequena isso
    mostra que o modelo percebeu similaridade
    '''
    def __init__(self, fitted_model, tracks_size):
        self.tracks_size = tracks_size
        self.fitted_model = fitted_model
        
    def evaluate(self, train_tracks, test_tracks):
        recommendations = self.fitted_model.recommend(train_tracks, n_tracks=self.tracks_size-2)
        return recommendations[recommendations.index.isin(test_tracks.index)]
    

In [17]:
cbe = ContentBasedEvaluator(sim4, len(tracks))
cbe.evaluate(
    pd.DataFrame([tracks.loc[anime_music]]), 
    pd.DataFrame([tracks.loc[classical_music_i]]))

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:4Znak3UnsXjRYQDSK0oP6V,1.366044,"Symphony No.5 in C minor, Op.67: 1. Allegro co...","{""(2wOqMjp9TyABvtHdOSOTUS,\""Ludwig van Beethov...",0.9,0.309,0.307,0.745,0.0929,0.764198,0.0443,0.402445


In [18]:
dataset = SpotifyPlaylistDataset()
dataset.read_playlist_formatted(100000)

Carregando dados da slice 0-999
Carregando dados da slice 100000-100999


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,ZAYN,spotify:track:3NdDpSvN911VPGivFlV5d0,spotify:artist:5ZsFI1h6hIdQRw2ti0hz81,I Don’t Wanna Live Forever (Fifty Shades Darke...,spotify:album:5MxXY7DbFMUiHFTPUabgJJ,245200,I Don’t Wanna Live Forever (Fifty Shades Darker)
1,1,Halsey,spotify:track:1uoL9GX45dTnmrWQ7Ii1Ft,spotify:artist:26VFTg2z8YR0cCuwLzESi2,Not Afraid Anymore,spotify:album:4HLA2Fsva7IJafJrhG5u52,226800,Not Afraid Anymore
2,2,Bryson Tiller,spotify:track:43PuMrRfbyyuz4QpZ3oAwN,spotify:artist:2EMAnMvWE2eb56ToJVfCWs,Exchange,spotify:album:6eZdwrhB97A3EYx9QppGfl,194613,T R A P S O U L
3,3,PARTYNEXTDOOR,spotify:track:1wZqJM5FGDEl3FjHDxDyQd,spotify:artist:2HPaUgqeutzr3jx5a9WyDV,Come and See Me (feat. Drake),spotify:album:2FXGUAESmG5l9YPrzWPvHI,235477,PARTYNEXTDOOR 3 (P3)
4,4,Bryson Tiller,spotify:track:3pXF1nA74528Edde4of9CC,spotify:artist:2EMAnMvWE2eb56ToJVfCWs,Don't,spotify:album:6eZdwrhB97A3EYx9QppGfl,198293,T R A P S O U L
5,5,J. Cole,spotify:track:6I6NX6tjGsxFAsIfGzY9lJ,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,Deja Vu,spotify:album:3CCnGldVQ90c26aFATC1PW,264746,4 Your Eyez Only
6,6,Joey Bada$$,spotify:track:4ntsD33Pgsgk8rUyZLLiUV,spotify:artist:2P5sC9cVZDToPxyomzF1UH,DEVASTATED,spotify:album:6swV0WUnPygRIMgEKn6Ige,207058,ALL-AMERIKKKAN BADA$$
7,7,Noah Cyrus,spotify:track:2BrzlUj1u1CtvaJDGIKpsP,spotify:artist:55fhWPvDiMpLnE4ZzNXZyW,Make Me (Cry),spotify:album:46yAYzRhSMPA44m0MzSr6g,242081,Make Me (Cry)


In [19]:
p1 = ['spotify:track:3NdDpSvN911VPGivFlV5d0',
       'spotify:track:1uoL9GX45dTnmrWQ7Ii1Ft',
       'spotify:track:43PuMrRfbyyuz4QpZ3oAwN',
       'spotify:track:1wZqJM5FGDEl3FjHDxDyQd',
       'spotify:track:3pXF1nA74528Edde4of9CC',
       'spotify:track:6I6NX6tjGsxFAsIfGzY9lJ',
       'spotify:track:4ntsD33Pgsgk8rUyZLLiUV',
       'spotify:track:2BrzlUj1u1CtvaJDGIKpsP']

In [20]:
rec_mean = Similarity() 
rec_mean.fit(tracks, music_ids)
rec_mean.recommend(tracks.loc[p1])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:0zU42XXyLpOKJ2y6A6egd5,0.036028,Need Armor,"{""(5fBcVfTWenqUDOYR5SjzEQ,\""Duwap Kaine\"")""}",0.190,0.691,0.459,0.000017,0.166,0.800518,0.154,0.519807
spotify:track:5rZKAY7OpzBY3m53a6Mi68,0.051123,Good Friends,"{""(0Bm5cFUEv8FGx1wEekHj0D,\""Jeffrey James\"")""}",0.192,0.679,0.495,0.000000,0.130,0.781988,0.149,0.552041
spotify:track:5okNoSDiGJPAD8HyEWae2f,0.054426,No Se Que Tengo Que No Se,"{""(2lHzW2pMhRYX3ohx5lZY8y,\""Los Pasteles Verde...",0.191,0.700,0.493,0.000000,0.133,0.765153,0.134,0.520287
spotify:track:0biinqb0iSI0G9e8ojzczU,0.064249,Tracy's Song,"{""(3KNIG74xSTc3dj0TRy7pGX,BLEU)""}",0.195,0.697,0.476,0.001050,0.174,0.808866,0.125,0.571694
spotify:track:0ivc6vKGX6OSJN4xZ6SwZd,0.064474,We Ain't Friends,"{""(2ZI5SpSI3z4jWeaFXr0WlC,\""Mickey Shiloh\"")"",...",0.164,0.698,0.457,0.000000,0.173,0.783944,0.117,0.480297
spotify:track:0U1KkFhpO1Y28ImIhrVwe0,0.065461,Dope Love (Feat. PeeWee Longway & Verse Simmonds),"{""(13y7CgLHjMVRMDqxdx0Xdo,\""Gucci Mane\"")"",""(0...",0.135,0.653,0.480,0.002840,0.146,0.767032,0.160,0.540176


# Modelo de Recomendação Content Based com Clusterização

In [21]:
class SimilarityMeans:
    def __init__(self, distance='euclidean'):
        self.distance = distance
        self.tracks = None
        self.id_tracks = None
        
    def fit(self, tracks, id_tracks):
        self.tracks = tracks
        self.id_tracks = id_tracks
        
    def clusterfy(self, user_tracks):
        best_score = np.inf
        best_model = None
        for i in range(2, len(user_tracks)) : 
            km = KMeans(n_clusters=i)
            km.fit(user_tracks)
            sc = silhouette_score(user_tracks, km.labels_)
            if (sc < best_score) :
                best_score = sc
                best_model = km
        return best_model
    
    def recommend_clusters(self, user_tracks, n_tracks=5):
        self.tracks = self.tracks.drop(index=user_tracks.index)
        self.id_tracks = self.id_tracks.drop(index=user_tracks.index)
        
        if len(user_tracks)==1:
            return [self.recommend(user_tracks[0], n_tracks)]
        
        model = self.clusterfy(user_tracks)
        clusters = model.cluster_centers_
        tracks = []
        for i in range(len(clusters)) :
            rec = self.recommend(clusters[i], n_tracks)
            tracks.append(rec)
        
        return tracks, pd.Series(model.labels_)
        
    def recommend(self, user_track, n_tracks=5):
        # calcula distancias e pega os indices das k músicas com menores distâncias
        dists = cdist([user_track], self.tracks, self.distance)[0]
        k_arg_dists = np.argpartition(dists, n_tracks)[:n_tracks]
        
        k_ids = self.id_tracks.iloc[k_arg_dists]
        k_info = self.tracks.iloc[k_arg_dists]
        k_dists = pd.DataFrame(dists[k_arg_dists], columns=['Distâncias'], index=self.tracks.index[k_arg_dists])
        
        # retorna um dataframe com informação do nome, artistas, distância e features ordenadas pela distância
        return pd.concat([k_dists, k_ids, k_info], axis=1).sort_values(by=['Distâncias'])

In [22]:
rec = SimilarityMeans(distance='euclidean')
rec.fit(tracks, music_ids)

In [23]:
tracks_to_recommend = tracks.loc[p1]
recommended, labels = rec.recommend_clusters(tracks_to_recommend)

In [24]:
for i,r in enumerate(recommended):
    print(f'Músicas pertecentes ao cluster {i+1}:')
    display(music_ids.loc[tracks_to_recommend[np.array(labels==i)].index])
    print(f'Recomendação para o cluster {i+1}:')
    display(r)

Músicas pertecentes ao cluster 1:


,name,artists
id,,
spotify:track:6I6NX6tjGsxFAsIfGzY9lJ,Deja Vu,"{""(6l3HvQ5sa6mXTsMTB19rO5,\""J. Cole\"")""}"


Recomendação para o cluster 1:


,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:2M80I6NoBN2mo6Lh3R9rln,0.090062,Lokote Diss (feat. Sicc2 Sicc G's),"{""(2pjKACDg4LkpZaaT9IyOky,\""Murder Squad\"")"",""...",0.346,0.656,0.486,0.000000,0.2150,0.734871,0.437,0.743647
spotify:track:36FPRDpKyP8hJxPhJs4G17,0.092802,Cliché,"{""(05uheFGPahVh2nHIZK1KuB,Jenyer)""}",0.327,0.593,0.431,0.000000,0.2100,0.803321,0.415,0.679159
spotify:track:1EXO9KgNVYD2bTYAEnnXy5,0.094764,Tis Is,"{""(6XFMQ3x86BHdNe7DCAox2y,\""Levi Star\"")""}",0.328,0.733,0.398,0.000000,0.1850,0.755541,0.373,0.719317
spotify:track:3nGMOOdzDMdzHp8g2VyuZl,0.100471,Summer Day,"{""(0Rb6PZXrcxvzixAwZwFmMu,\""Soom T\"")""}",0.340,0.654,0.424,0.000008,0.0903,0.754371,0.417,0.719241
spotify:track:3gY7rOLz3hBNWFgDsLpmyj,0.107867,One More Time,"{""(1GO9LmsSPXbqZjWkfViwIl,\""Mike Pinto\"")""}",0.382,0.646,0.334,0.000000,0.1570,0.742680,0.453,0.675663


Músicas pertecentes ao cluster 2:


...

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:3RnsXEbwWdzNGnVJcxZ3mb,0.075085,It Hurts So Good - 1,"{""(27XI002zivG9xuzrZ9dIb1,\""Millie Jackson\"")""}",0.0945,0.496,0.466,0.000000,0.1050,0.768680,0.140,0.616664
spotify:track:6UGclhMtFvVPnaw2jy5lcv,0.078128,Shake It Baby,"{""(1yNOfXGQNGjAynk77wv85x,\""John Lee Hooker\"")""}",0.0692,0.540,0.424,0.000149,0.0962,0.719036,0.169,0.669663
spotify:track:4KCoVMB1MSoSl9YzQipTcb,0.080104,La Gallera,"{""(6jIvSVTvBMdlsCWqTQJKbJ,\""Sofia Rei\"")""}",0.1260,0.506,0.421,0.000033,0.1010,0.778599,0.147,0.691832
spotify:track:2rQeGKmFMKtXjSXYOWq70Q,0.083087,Buona Sera,"{""(52lBOxCxbJg0ttXEW9CQpW,\""Louis Prima\"")""}",0.1070,0.471,0.428,0.000001,0.0979,0.746993,0.197,0.687876
spotify:track:4URCBE0Z7uPxHQUC0ITfd5,0.083934,Just Tryna,"{""(2y246nnP9pQT0E6v3ZMMOO,\""Night Lovell\"")"",""...",0.1220,0.568,0.420,0.000017,0.1960,0.730773,0.198,0.638941


Músicas pertecentes ao cluster 7:


,name,artists
id,,
spotify:track:1wZqJM5FGDEl3FjHDxDyQd,Come and See Me (feat. Drake),"{""(2HPaUgqeutzr3jx5a9WyDV,PARTYNEXTDOOR)"",""(3T..."


Recomendação para o cluster 7:


,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:32Cq0UDK17kqQCl7s5SwiY,0.036195,Por Tu Amor,"{""(0ZCO8oVkMj897cKgFH7fRW,\""Los Ángeles Azules...",0.0582,0.734,0.364,0.000000,0.0921,0.786208,0.0602,0.324533
spotify:track:1sYyAPxndxAGGbc46v1Uqx,0.037995,Rhythm Streets,"{""(6MxlVTY6PmY8Nyn16fvxtb,\""Slightly Stoopid\"")""}",0.0453,0.731,0.327,0.010800,0.1020,0.768249,0.0455,0.328133
spotify:track:00zAgZtI5qBYnsIiKkIMmo,0.039910,Since I Fell For You,"{""(312pFk2xpEwYWLGPehc4Nu,\""John Holt\"")""}",0.0628,0.737,0.353,0.000495,0.0799,0.765815,0.0384,0.321685
spotify:track:5WIjamF8M8BCmpRNSDYfdT,0.040151,Easy,"{""(2LMtb7Pl4o1lBqKug4o5l7,\""Lukie D\"")""}",0.0401,0.716,0.321,0.000000,0.1080,0.758175,0.0556,0.324253
spotify:track:5HtfMwWF15H3k4McsCwhbS,0.041886,Use Me,"{""(0bry27FuGIjWBs0sZ2JC2C,\""Kevin James O'Brie...",0.0537,0.729,0.320,0.000770,0.1000,0.749195,0.0602,0.319741
